# 使用 Cognee 建立具持久記憶的 AI 智能代理

此筆記本展示如何使用 [**cognee**](https://www.cognee.ai/) 建立具備高級記憶功能的智能 AI 代理。Cognee 是一個開源的 AI 記憶系統，結合知識圖譜、語義搜索及會話管理，打造具上下文感知能力的 AI 系統。

## 🎯 學習目標

完成本教程後，您將了解如何：
- **建立基於嵌入的知識圖譜**：將非結構化文本轉化為結構化、可查詢的知識
- **實現會話記憶**：創建多輪對話並自動保留上下文
- **持久化對話**：選擇性地將重要互動存儲於長期記憶中以供未來參考
- **使用自然語言查詢**：在新對話中訪問並利用歷史上下文
- **可視化記憶**：探索代理知識圖譜中的關係


## 🏗️ 你將會建立什麼

在這個教學中，我們會建立一個具有持久記憶的 **編程助手**，它能夠：

### 1. **知識庫建構**
   - 收集開發者的個人資料及專業資訊
   - 處理 Python 編程原則及最佳實踐
   - 儲存開發者與 AI 助手之間的歷史對話

### 2. **會話感知功能**
   - 在同一個會話中保持多個問題的上下文
   - 自動緩存每個問題及答案以便高效檢索
   - 根據對話歷史提供連貫且有上下文的回應

### 3. **長期記憶**
   - 將重要的對話保存到長期記憶中
   - 從知識庫及過往會話中檢索相關記憶以輔助新的互動
   - 建立一個隨時間增長的知識庫，持續改進

### 4. **智能記憶檢索**
   - 使用圖形感知語義搜索來尋找所有儲存知識中的相關資訊
   - 按數據子組（例如開發者資訊 vs. 原則）篩選搜索結果
   - 結合多個數據來源以提供全面的答案


## 📋 先決條件及設定

### 系統需求

在開始之前，請確保你已準備好以下項目：

1. **Python 環境**
   - Python 3.9 或更高版本
   - 建議使用虛擬環境

2. **Redis 快取**（用於會話管理）
   - 本地 Redis：`docker run -d -p 6379:6379 redis`
   - 或使用托管的 Redis 服務

3. **LLM API 訪問**
   - OpenAI API 金鑰或其他供應商（請參閱[文件](https://docs.cognee.ai/setup-configuration/llm-providers)）

4. **資料庫配置**
   - 預設情況下無需配置。Cognee 使用基於文件的資料庫（LanceDB 和 Kuzu）
   - 可選地，你可以設置 Azure AI Search 作為向量存儲（請參閱[文件](https://github.com/topoteretes/cognee-community/tree/main/packages/vector/azureaisearch)）

### 環境配置

在你的項目目錄中建立 `.env` 文件，並包含以下變數：

```ini
# LLM Configuration (Required)
LLM_API_KEY=your-openai-api-key-here

# Cache Configuration (Required for Sessions)
CACHING=true  # Must be enabled for session history

```


## 🏛️ Cognee 記憶架構的理解

### Cognee 的運作方式

Cognee 提供了一個超越簡單鍵值存儲的高級記憶系統：

```
┌──────────────────────────┐
│      30+ data sources    │
└───────────┬──────────────┘
            │
            ▼
┌──────────────────────────────────────────┐
│  Dynamically evolving memory layers      │
│                                          │
│  ┌────────────────────────────────────┐  │
│  │ Knowledge Graph in Graph Database  │  │
│  └────────────────────────────────────┘  │
│  ┌────────────────────────────────────┐  │
│  │ Embeddings in Vector Store         │  │
│  │   (e.g., Azure AI Search)          │  │
│  └────────────────────────────────────┘  │
└───────────┬──────────────────────────────┘
            │                      ▲   
            ▼                      │(optional)
┌────────────────┐           ┌────────────────┐
│     cognee     │(optional) │ Cognee Session │
│    retrievers  │──────────▶│     Cache      │
│                │           │    (Redis)     │
└───────┬────────┘           └────────────────┘
        ▲
        │
┌──────────────────────────┐
│          Agents          │
└──────────────────────────┘

```

### 主要組成部分：

1. **知識圖譜**：存儲實體、關係及語義連結
2. **向量嵌入**：支持對所有存儲信息進行語義搜索
3. **會話緩存**：在會話內及跨會話維持對話上下文
4. **節點集**：將數據組織成邏輯分類以便於目標檢索

### 本教程中的記憶類型：

- **持久記憶**：知識圖譜中的長期存儲
- **會話記憶**：Redis 緩存中的臨時對話上下文
- **語義記憶**：基於向量的相似性搜索，涵蓋所有數據


## 📦 安裝所需套件

安裝 Cognee，並支援 Redis 作為會話管理：


In [ ]:
!pip install --quiet "cognee[redis]==0.4.0"

## 🔧 初始化環境及載入庫

請確保：
1. Redis 已啟動（例如使用 Docker：`docker run -d -p 6379:6379 redis`）
2. 在導入快取模組之前已設定環境變數
3. 如有需要，重新啟動內核並按順序執行所有單元格

以下單元格將會：
1. 從 `.env` 載入環境變數
2. 使用你的 LLM 設定配置 Cognee
3. 啟用快取以管理會話
4. 驗證所有元件是否已正確連接


In [ ]:
import os
from pathlib import Path

from dotenv import load_dotenv

# Load environment variables
load_dotenv()

# cognee Configuration
os.environ["LLM_API_KEY"] = os.getenv("LLM_API_KEY", None)
os.environ["CACHING"] = os.getenv("CACHING", "true")


import cognee

print(f"Cognee version: {cognee.__version__}")
print(f"CACHING: {os.environ.get('CACHING')}")
print(f"LLM_API_KEY: {os.environ.get('LLM_API_KEY')}")

## 📁 配置存儲目錄

Cognee 使用兩個獨立的目錄進行操作：
- **數據根目錄**：存儲已導入的文件和處理後的數據
- **系統根目錄**：包含知識圖譜數據庫和系統元數據

我們將為本教程創建獨立的目錄，如下所示：


In [ ]:
DATA_ROOT = Path('.data_storage').resolve()
SYSTEM_ROOT = Path('.cognee_system').resolve()

DATA_ROOT.mkdir(parents=True, exist_ok=True)
SYSTEM_ROOT.mkdir(parents=True, exist_ok=True)

cognee.config.data_root_directory(str(DATA_ROOT))
cognee.config.system_root_directory(str(SYSTEM_ROOT))

print(f"Data root: {DATA_ROOT}")
print(f"System root: {SYSTEM_ROOT}")

## 🧹 重置記憶狀態

在我們開始建立記憶系統之前，先確保我們從頭開始。

> 💡 **提示**：如果你想在之後使用這個筆記本時保留之前運行的記憶，可以跳過這一步。


In [ ]:
await cognee.prune.prune_data()
await cognee.prune.prune_system(metadata=True)
print('Cleared previous Cognee state.')

## 📚 第 1 部分：建立知識庫

### 我們的開發者助手的數據來源

我們將引入三種類型的數據來建立一個全面的知識庫：

1. **開發者檔案**：個人的專業知識和技術背景  
2. **Python 最佳實踐**：Python 之禪及實用指引  
3. **歷史對話**：開發者與 AI 助手之間的過往問答記錄  

這些多樣化的數據能讓我們的代理做到以下幾點：  
- 理解用戶的技術背景  
- 在建議中應用最佳實踐  
- 從以往成功的互動中學習  


In [ ]:
developer_intro = (
  "Hi, I'm an AI/Backend engineer. "
  "I build FastAPI services with Pydantic, heavy asyncio/aiohttp pipelines, "
  "and production testing via pytest-asyncio. "
  "I've shipped low-latency APIs on AWS, Azure, and GoogleCloud."
)

python_zen_principles = (
  """
    # The Zen of Python: Practical Guide

    ## Overview
    Use these principles as a checklist during design, coding, and reviews.

    ## Key Principles With Guidance

    ### 1. Beautiful is better than ugly
    Prefer descriptive names, clear structure, and consistent formatting.

    ### 2. Explicit is better than implicit
    Be clear about behavior, imports, and types.
    ```python
    from datetime import datetime, timedelta

    def get_future_date(days_ahead: int) -> datetime:
        return datetime.now() + timedelta(days=days_ahead)
    ```

    ### 3. Simple is better than complex
    Choose straightforward solutions first.

    ### 4. Complex is better than complicated
    When complexity is needed, organize it with clear abstractions.

    ### 5. Flat is better than nested
    Use early returns to reduce indentation.

    ## Modern Python Tie-ins
    - Type hints reinforce explicitness
    - Context managers enforce safe resource handling
    - Dataclasses improve readability for data containers

    ## Quick Review Checklist
    - Is it readable and explicit?
    - Is this the simplest working solution?
    - Are errors explicit and logged?
    - Are modules/namespaces used appropriately?
  """
)

human_agent_conversations = (
  """
  "conversations": [
      {
        "id": "conv_001",
        "timestamp": "2024-01-15T10:30:00Z",
        "topic": "async/await patterns",
        "user_query": "I'm building a web scraper that needs to handle thousands of URLs concurrently. What's the best way to structure this with asyncio?",
        "assistant_response": "Use asyncio with aiohttp, a semaphore to cap concurrency, TCPConnector for connection pooling, context managers for session lifecycle, and robust exception handling for failed requests.",
        "code_context": {
          "file": "scraper.py",
          "language": "python",
          "patterns_discussed": ["async/await", "context_managers", "semaphores", "aiohttp", "error_handling"]
        },
        "follow_up_questions": [
          "How do I add retry logic for failed requests?",
          "What's the best way to parse the scraped HTML content?"
        ]
      },
      {
        "id": "conv_002",
        "timestamp": "2024-01-16T14:20:00Z",
        "topic": "dataclass vs pydantic",
        "user_query": "When should I use dataclasses vs Pydantic models? I'm building an API and need to handle user input validation.",
        "assistant_response": "For API input/output, prefer Pydantic: it provides runtime validation, type coercion, JSON serialization, enums for roles, field constraints, and custom validators; integrates cleanly with FastAPI for automatic request validation and error reporting.",
        "code_context": {
          "file": "models.py",
          "language": "python",
          "patterns_discussed": ["pydantic", "dataclasses", "validation", "fastapi", "type_hints", "enums"]
        },
        "follow_up_questions": [
          "How do I handle nested validation with Pydantic?",
          "Can I use Pydantic with SQLAlchemy models?"
        ]
      },
      {
        "id": "conv_003",
        "timestamp": "2024-01-17T09:15:00Z",
        "topic": "testing patterns",
        "user_query": "I'm struggling with testing async code and database interactions. What's the best approach for pytest with async functions?",
        "assistant_response": "Recommended using pytest-asyncio, async fixtures, and an isolated test database or mocks to reliably test async functions and database interactions in FastAPI.",
        "code_context": {
          "file": "test_users.py",
          "language": "python",
          "patterns_discussed": ["pytest", "async_testing", "fixtures", "mocking", "database_testing", "fastapi_testing"]
        },
        "follow_up_questions": [
          "How do I test WebSocket connections?",
          "What's the best way to test database migrations?"
        ]
      },
      {
        "id": "conv_004",
        "timestamp": "2024-01-18T16:45:00Z",
        "topic": "performance optimization",
        "user_query": "My FastAPI app is getting slow with large datasets. How can I optimize database queries and response times?",
        "assistant_response": "Suggested optimizing database queries (indexes, pagination, selecting only needed columns), adding caching, streaming or chunked responses for large datasets, background tasks for heavy work, and monitoring to find bottlenecks.",
        "code_context": {
          "file": "optimizations.py",
          "language": "python",
          "patterns_discussed": ["performance_optimization", "caching", "database_optimization", "async_patterns", "monitoring"]
        },
        "follow_up_questions": [
          "How do I implement database connection pooling properly?",
          "What's the best way to handle memory usage with large datasets?"
        ]
      },
      {
        "id": "conv_005",
        "timestamp": "2024-01-19T11:30:00Z",
        "topic": "error handling and logging",
        "user_query": "I need to implement proper error handling and logging across my Python application. What's the best approach for production-ready error management?",
        "assistant_response": "Proposed centralized error handling with custom exceptions, structured logging, FastAPI middleware or decorators, and integration points for external monitoring/alerting tools.",
        "code_context": {
          "file": "error_handling.py",
          "language": "python",
          "patterns_discussed": ["error_handling", "logging", "exceptions", "middleware", "decorators", "fastapi"]
        },
        "follow_up_questions": [
          "How do I integrate this with external monitoring tools like Sentry?",
          "What's the best way to handle errors in background tasks?"
        ]
      }
    ],
    "metadata": {
      "total_conversations": 5,
      "date_range": "2024-01-15 to 2024-01-19",
      "topics_covered": [
        "async/await patterns",
        "dataclass vs pydantic",
        "testing patterns",
        "performance optimization",
        "error handling and logging"
      ],
      "code_patterns_discussed": [
        "asyncio", "aiohttp", "semaphores", "context_managers",
        "pydantic", "fastapi", "type_hints", "validation",
        "pytest", "async_testing", "fixtures", "mocking",
        "performance_optimization", "caching", "database_optimization",
        "error_handling", "logging", "exceptions", "middleware"
      ],
      "difficulty_levels": {
        "beginner": 1,
        "intermediate": 2,
        "advanced": 2
      }
    }
  """
)

## 🔄 將數據轉化為知識圖譜

現在我們將把原始文本轉化為結構化的記憶。這個過程包括：

1. **將數據添加到 NodeSets**：將信息組織到邏輯分類中
   - `developer_data`：開發者檔案和對話
   - `principles_data`：Python 的最佳實踐和指引

2. **運行 Cognify Pipeline**：提取實體、關係並創建嵌入
   - 識別關鍵概念
   - 在相關信息之間創建語義連接
   - 生成向量嵌入

這可能需要一些時間，因為 LLM 正在處理文本並構建圖譜結構：


In [ ]:
await cognee.add(developer_intro, node_set=["developer_data"])
await cognee.add(human_agent_conversations, node_set=["developer_data"])
await cognee.add(python_zen_principles, node_set=["principles_data"])

await cognee.cognify()

## 📊 視覺化知識圖譜

讓我們來探索知識圖譜的結構。視覺化內容顯示：
- **節點**：從文本中提取的實體（概念、技術、人員）
- **邊**：實體之間的關係和連接
- **群集**：按語義相似性分組的相關概念

在瀏覽器中打開生成的 HTML 文件，互動式地探索圖譜：


In [ ]:
from cognee import visualize_graph
await visualize_graph('./visualization_1.html')

## 🧠 用 Memify 豐富記憶

`memify()` 函數會分析知識圖譜，並針對數據生成智能規則。這個過程：
- 識別模式和最佳實踐
- 根據內容創建可行的指引
- 建立不同知識領域之間的關係

這些規則幫助代理在回答問題時作出更明智的決定。捕捉第二次的可視化圖表，能幫助你比較圖譜在豐富後如何變得更密集。


In [ ]:
await cognee.memify()

await visualize_graph('./visualization_2.html')

## 🔍 第 2 部分：智能記憶檢索

### 示範 1：跨文件知識整合

現在我們已經建立了知識圖譜，讓我們測試 Cognee 如何結合多個來源的資訊來回答複雜問題。

第一個查詢展示了：
- **語義理解**：即使未明確提及，也能找到相關概念
- **交叉參照**：結合開發者檔案與 Python 原則
- **情境推理**：將最佳實踐應用於特定實現

### 示範 2：使用 NodeSets 的篩選搜尋

第二個查詢展示如何針對知識圖譜的特定子集進行搜尋：
- 使用 `node_name` 參數僅在 `principles_data` 中搜尋
- 提供來自特定知識領域的聚焦答案
- 適用於需要領域專屬資訊的情況


In [ ]:
# demonstrate cross-document knowledge retrieval from multiple data sources
from cognee.modules.search.types import SearchType

results = await cognee.search(
    query_text="How does my AsyncWebScraper implementation align with Python's design principles?",
    query_type=SearchType.GRAPH_COMPLETION,
)
print("Python Pattern Analysis:", results)

# demonstrate filtered search using NodeSet to query only specific subsets of memory
from cognee.modules.engine.models.node_set import NodeSet
results = await cognee.search(
    query_text="How should variables be named?",
    query_type=SearchType.GRAPH_COMPLETION,
    node_type=NodeSet,
    node_name=["principles_data"],
)
print("Filtered search result:", results)

## 🔐 第三部分：會話管理設置

### 啟用對話記憶

會話管理對於在多次互動中保持上下文非常重要。在這裡我們會：

1. **初始化用戶上下文**：創建或檢索用戶檔案以進行會話追蹤  
2. **配置緩存引擎**：連接到 Redis 以存儲對話歷史記錄  
3. **啟用會話變數**：設置在查詢間持續存在的上下文變數  

> ⚠️ **重要**：這需要 Redis 正在運行，並且環境中設置了 `CACHING=true`


In [ ]:
from cognee.modules.users.methods import get_default_user
from cognee.context_global_variables import set_session_user_context_variable 
from cognee.infrastructure.databases.cache import get_cache_engine

user = await get_default_user()
await set_session_user_context_variable(user)
print(f"Using user id: {getattr(user, 'id', 'unknown')}")

cache_engine = get_cache_engine()
if cache_engine is None:
    raise RuntimeError('Cache engine is not available. Double-check your cache configuration.')
print('Session cache is ready.')


## 🛠️ 輔助函數：查看會話歷史記錄

此實用函數讓我們可以檢視儲存在 Redis 中的會話歷史記錄。這對以下情況非常有用：
- 偵錯會話管理
- 驗證會話是否已被緩存
- 瞭解代理可用的上下文


In [ ]:
async def show_history(session_id: str) -> None:
    # Let's check the cache directly
    cache_engine = get_cache_engine()
    if cache_engine:
        # Try to get history directly from cache
        user_id = str(user.id) if hasattr(user, 'id') else None
        if user_id:
            history_entries = await cache_engine.get_latest_qa(user_id, session_id, last_n=10)
            print(f"\nDirect cache query for user_id={user_id}, session_id={session_id}:")
            print(f"Found {len(history_entries)} entries")
            if history_entries:
                for i, entry in enumerate(history_entries, 1):
                    print(f"\nEntry {i}:")
                    print(f"  Question: {entry.get('question', 'N/A')[:100]}...")
                    print(f"  Answer: {entry.get('answer', 'N/A')[:100]}...")
        else:
            print("No user_id available")


## 第一節：非同步支援實驗室 — 第一個問題

開始 `async-support-lab` 的環節，詢問適合大量網頁爬取的具備遙測友好的 asyncio 模式。圖表已經了解 asyncio、aiohttp 和監控方法，因此回應應該反映之前的對話，同時針對新問題進行調整。


In [ ]:
session_1 = "async-support-lab"

result = await cognee.search(
    query_type=SearchType.GRAPH_COMPLETION,
    query_text="I'm building a web scraper that hits thousands of URLs concurrently. What's a reliable asyncio pattern with telemetry?",
    session_id=session_1
)

## 檢查第一節記憶體在首次交流後的狀況

在首次提問後立即執行 `show_history(session_1)`，可以確認 Cognee 已將提示和完成內容寫入 Redis。你應該會看到一個包含並行指導的記錄。


In [ ]:
await show_history(session_1)

## 第 1 節：跟進數據模型

接下來我們問：「什麼時候應該選擇 dataclasses 而不是 Pydantic？」使用相同的會話 ID。Cognee 應該結合 Python 的原則以及之前有關 FastAPI 的對話，提供細緻的建議——展示出在命名會話中，內容是可以延續的。


In [ ]:
result = await cognee.search(
    query_type=SearchType.GRAPH_COMPLETION,
    query_text="When should I pick dataclasses versus Pydantic for this work?",
    session_id=session_1
)

## 確認 Session 1 歷史記錄包含兩個回合

再次執行 `show_history(session_1)` 應該會顯示兩個問答條目。這與 Mem0 實驗室的「記憶重播」步驟相符，並證明額外的回合會延續相同的對話記錄。


In [ ]:
await show_history(session_1)

## 第2節：設計檢討討論串 — 新會話

為了展示討論串之間的隔離，我們啟動了 `design-review-session`，並請求有關事件檢討的記錄指引。即使底層的知識庫相同，新的會話 ID 會將記錄分開。


In [ ]:
session_2 = "design-review-session"

result = await cognee.search(
    query_type=SearchType.GRAPH_COMPLETION,
    query_text="We're drafting logging guidance for incident reviews. Capture the key principles please.",
    session_id=session_2
)

## 檢討會議 2 歷史記錄

`show_history(session_2)` 應該只列出設計檢討的提示/回應對比。將其與會議 1 比較，以突顯 Cognee 如何在重用共享知識圖譜的同時，保持獨立的會議記錄。


In [ ]:
await show_history(session_2)

## 摘要

恭喜你！你剛剛為你的編程助手添加了一個由 Cognee 驅動的真正長期記憶層。

在這個教程中，你將原始的開發者內容（代碼、文檔、聊天）轉化為一個圖形 + 向量記憶，讓你的代理可以進行搜索、推理，並持續改進。

你學到了什麼

1. **從原始文本到 AI 記憶**：了解 Cognee 如何吸收非結構化數據，並通過結合向量 + 知識圖譜架構，將其轉化為智能、可搜索的記憶。

2. **使用 memify 豐富圖譜**：學習如何超越基本的圖譜創建，使用 memify 在現有圖譜上添加衍生事實和更豐富的關係。

3. **多種搜索策略**：了解如何根據代理的需求，使用不同的搜索類型（圖譜感知問答、RAG 風格補全、洞察、原始片段、代碼搜索等）來查詢記憶。

4. **視覺化探索**：學習如何使用圖譜可視化和 Cognee UI 檢查和調試 Cognee 建立的內容，讓你能夠真正看到知識的結構。

5. **會話感知記憶**：了解如何結合每次會話的上下文與持久的語義記憶，讓代理能夠在多次運行中記住內容，而不會在用戶之間洩漏信息。


## 關鍵重點
1. 以嵌入支持的知識圖譜作為記憶

    - **結構化理解**：Cognee 結合了向量存儲和圖譜存儲，讓您的數據既可以按意義搜索，也可以按關係連接。Cognee 預設使用基於文件的數據庫（LanceDB 用於向量存儲，Kuzu 用於圖譜數據庫）。

    - **關係感知檢索**：答案不僅可以基於「相似文本」，還可以基於實體之間的關係。

    - **活記憶**：記憶層會隨著時間演變、增長，並保持作為一個連接的圖譜可供查詢。

2. 搜索與推理模式
    - **混合檢索**：搜索結合了向量相似性、圖譜結構和 LLM 推理，從原始片段查詢到基於圖譜的問題回答。

    - **根據需求選擇模式**：當需要自然語言答案時使用完成式模式，當代理需要原始上下文或進行自主推理時使用片段/摘要/圖譜模式。

3. 個性化、會話感知的代理
    - **會話上下文 + 長期記憶**：Cognee 將短期「線索」上下文與長期的用戶或組織層級記憶分開。

## 真實世界應用

1. **垂直 AI 代理**

    使用此筆記本中的模式，為基於 Cognee 的檢索和推理核心的領域智能助手提供支持：

- **開發者助手**：代碼審查、事件分析和架構助手，能夠遍歷代碼、API、設計文檔和票據，作為一個單一的記憶圖譜。

- **面向客戶的助手**：支持或成功代理，能從產品文檔、常見問題解答、CRM 記錄和過去的票據中提取信息，並提供基於圖譜檢索的引用答案。

- **內部專家助手**：政策、法律或安全助手，能夠基於互相關聯的規則、指南和歷史決策進行推理，而不是孤立的 PDF。

    Cognee 明確定位為 AI 代理的持久、準確記憶，提供一個活的知識圖譜，作為代理的後端，取代臨時組合的向量存儲和自定義圖譜代碼。

2. **將數據孤島統一為一個記憶**

    同樣的方法也能幫助您在分散的來源之間建立統一的記憶層：

- **從孤島到一個圖譜**：將結構化（例如數據庫）和非結構化數據（例如文檔、聊天）導入一個由嵌入支持的單一圖譜，而不是為每個系統建立單獨的索引。

- **跨來源推理並提供引用**：在所有數據上運行多步推理——通過圖譜「連接」日誌、指標和文檔——並仍然返回有根據的答案和來源。

- **知識中心**：在銀行或教育等領域，Cognee 已被用於統一 PDF、內部系統和應用數據，形成一個帶有向量的知識圖譜，讓代理能夠以精確且有引用的上下文回答問題。

## 下一步

您已經實現了核心記憶循環。以下是您可以自行嘗試的自然擴展（詳情請參閱 [Cognee 文檔](https://docs.cognee.ai/)）：

1. **嘗試時間感知**：啟用時間認知功能，從文本中提取事件和時間戳。

2. **引入基於本體的推理**：為您的領域定義 OWL 本體。使用 Cognee 的本體支持，讓提取的實體和關係基於該架構，提升圖譜質量和領域特定答案。

3. **添加反饋循環**：讓 Cognee 根據真實用戶反饋調整圖譜邊緣權重，使檢索隨時間改進而非保持靜態。

4. **調整個性化與會話行為**：使用用戶 ID、租戶和數據集，為每個人或團隊提供他們自己的共享記憶引擎視圖。

5. **擴展到更複雜的代理**：將 Cognee 插入代理框架，構建共享同一記憶層的多代理系統。*Microsoft Agent Framework x Cognee 插件即將推出。*


---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**免責聲明**：  
此文件已使用人工智能翻譯服務 [Co-op Translator](https://github.com/Azure/co-op-translator) 進行翻譯。雖然我們致力於提供準確的翻譯，但請注意，自動翻譯可能包含錯誤或不準確之處。原始語言的文件應被視為權威來源。對於重要資訊，建議使用專業的人手翻譯。我們對因使用此翻譯而引起的任何誤解或誤釋不承擔責任。
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
